## E0 : Installer les dépendances

In [2]:
pip install --upgrade pip
pip install Flask PyPDF2 pdfplumber ipywidgets
!pip install torch transformers datasets
!pip install PyMuPDF

SyntaxError: invalid syntax (1957491074.py, line 1)

## BERT

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch

### Exemple de modèle de dataset

In [6]:
# Dataset personnalisé
class ProceduralViceDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

### Charger le tokenizer et le modèle BERT pré-entrainé

In [8]:
# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Créer un modèle de classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Vérifier le périphérique (GPU/CPU/MPS)
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### Préparer les données (exemple fictif)

In [9]:
import random

# Fonction pour générer un texte de procédure
def generate_text():
    # Scénarios de vices de procédure possibles
    vices = [
        "Le juge a omis de notifier l'audience aux parties concernées.",
        "Le jugement a été rendu sans que l'accusé ait eu la possibilité de se défendre.",
        "La décision a été prise sans respecter les délais légaux.",
        "L'appel a été jugé sans que les preuves aient été correctement examinées.",
        "Le procureur a omis de présenter des preuves cruciales au tribunal.",
        "Il n'y a pas eu de notification de l'audience au représentant de la défense."
    ]
    
    # Scénarios sans vice de procédure
    no_vices = [
        "Le jugement a été rendu après que l'accusé ait eu la possibilité de se défendre.",
        "Les parties ont été dûment informées de la date de l'audience.",
        "L'appel a été jugé dans les délais légaux.",
        "Les preuves ont été minutieusement examinées avant la décision.",
        "L'accusé a eu la possibilité de présenter tous les arguments lors de l'audience.",
        "L'audience a eu lieu selon les normes et procédures établies."
    ]
    
    # Choisir aléatoirement un scénario avec ou sans vice de procédure
    if random.choice([True, False]):
        text = random.choice(vices)
        label = 1  # vice de procédure
    else:
        text = random.choice(no_vices)
        label = 0  # pas de vice de procédure
    
    return text, label

# Générer 100 exemples
texts = []
labels = []

for _ in range(100):
    text, label = generate_text()
    texts.append(text)
    labels.append(label)

# Afficher un aperçu des premières valeurs
print("Premiers textes:", texts[:5])  # Afficher les 5 premiers textes
print("Premiers labels:", labels[:5])  # Afficher les 5 premiers labels


Premiers textes: ["L'appel a été jugé dans les délais légaux.", "Le juge a omis de notifier l'audience aux parties concernées.", "L'audience a eu lieu selon les normes et procédures établies.", 'Les preuves ont été minutieusement examinées avant la décision.', 'Les preuves ont été minutieusement examinées avant la décision.']
Premiers labels: [0, 1, 0, 0, 0]


In [ ]:
#texts = ["Le texte 1 avec un vice de procédure", "Texte 2 sans vice de procédure"]
#labels = [1, 0]  # 1 = vice de procédure, 0 = pas de vice de procédure


In [15]:
# Créer le Dataset
dataset = ProceduralViceDataset(texts, labels, tokenizer, max_len=512)

# Diviser en train et validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Créer un Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tuning du modèle
trainer.train()


model.save_pretrained("./results/checkpoint-15/")
tokenizer.save_pretrained("./results/checkpoint-15/")


# Fonction de prédiction
def predict(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Déplacer les tensors sur le même périphérique que le modèle
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=-1)
        
    return "Vice de procédure détecté" if prediction == 1 else "Pas de vice de procédure"

# Exemple de prédiction
texte_a_predire = "La décision a été prise sans respecter les délais légaux."
print(predict(texte_a_predire))

  0%|          | 0/15 [00:00<?, ?it/s]

{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 1.0000000000000002e-06, 'epoch': 2.0}
{'train_runtime': 267.2913, 'train_samples_per_second': 0.898, 'train_steps_per_second': 0.056, 'train_loss': 0.0, 'epoch': 3.0}
Pas de vice de procédure


In [16]:
texte_a_predire = "La décision a été prise sans respecter les délais légaux.."
print(predict(texte_a_predire))

Pas de vice de procédure
